#### Na kreslenie budeme používať modul `plotly`, konkrétne `plotly.express`, čo aj vám odporúčame.

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np  # pre vektorove operacie

In [ ]:
# nacitame vycistenu frejmu, len stlpce, co pouzijeme v tomto NB
cols = ['pick_dt', 'passengers', 'fare']
df = pd.read_parquet('data/nyc_taxi310k.parq', columns=cols)

In [ ]:
df.min(), df.max() # min. a max. vo vsetkych stlpcov

### Všetky dáta sú z januára 2015. 
#### Zoberme nástupy (pick). Chceme grafy po jednotlivých dňoch. Počet pasažierov, počet jázd, zarobené peniaze.

In [ ]:
# pocet cestujucich, najskor bez 'alias'
df_days = pd.DataFrame(df.groupby(df['pick_dt'].dt.day)['passengers'].sum().astype('float32'))

In [ ]:
# pred a po alias ukazat
df_days['pick_day']=df_days.index

In [ ]:
df_days

### Nakreslíme stĺpcový graf, na osi x budú dni (1, 2, .... 31. jan.), na osi y počty cestujúcich

In [ ]:
pass_plot = px.bar(df_days, x='pick_day', y='passengers', labels={'pick_day':'Dni', 'pass_count': 'Cestujúci'})
# dni v mesiaci chceme pre kazdy den
pass_plot.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 32))));

In [ ]:
pass_plot
# ukazat hover a nastroje plotly

## Čo za katastrófa sa stala 27. januára? Strašný pokles oproti iným dňom.
#### Vidíme nejakú (približnú) periodicitu v dátach?
### Podobné grafy by sme chceli pre počty jázd a pre denné zárobky.

In [ ]:
# zrobime frejmu, kde budu vsetky tie tri veliciny agregovane podla dni - ako hore, len zoznam troch stlpcov namiesto jedneho

# df_days = 
groupped = df.groupby(df['pick_dt'].dt.day)
df_days['pass_count'] = groupped['passengers'].sum()
df_days['day_fares'] = groupped['fare'].sum() 
df_days['fares_count'] = groupped['fare'].count()

In [ ]:
# ake asi budu skutocne trzby, ked my mame vzorku 310 000 zapisov z 12 milionov
df_days = df_days.drop(columns='passengers')

In [ ]:
# mozeme nakreslit kombinovany graf pre pocet jazd a pocet cestujucich (malo by to byt zhruba radovo rovnake)

pass_fares_plot = px.bar(df_days, x='pick_day', y=['pass_count', 'fares_count'], barmode='group',)
                         # labels={'pick_day':'Dni', 'value': 'Hodnoty', 'variable': 'Premenná'})
# zobrazenie sa nemiesa s jeho rozpolozenim (layout)    
# pass_fares_plot.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 32))))
pass_fares_plot

### Grafy pre tie isté veličiny, no podľa hodín - to isté, len grupovanie podľa hodín
Jediná funkcia pre datafrejmu a aj pre graf.

In [ ]:
def monthly_frame(frm, day=True):      # nazvy stplcov v datafrejme mozu byt aj po slovensky :-)
    groupped = frm.groupby(frm['pick_dt'].dt.day) if day else\
               frm.groupby(frm['pick_dt'].dt.hour)         
    column = 'pick_day' if day else 'pick_hour'
    df_month = pd.DataFrame(groupped['passengers'].sum().astype('float32')).rename(columns={'passengers': 'Cestujúci'})
    df_month['Platby'] = groupped['fare'].sum() 
    df_month['Jazdy'] = groupped['fare'].count()
    df_month[column] = df_month.index   # pick_dt sa tu nehodi, bude pick_day alebo pick_hour
    return df_month

# V aplikacii budeme mat radiobox s volbami ['Podľa dní','Podľa hodín', 'Dni v týždni (nástupy)']

def monthly_plot(dhc):
    day = (dhc == 'Podľa dní')
    xcol = 'pick_day' if day else 'pick_hour'
    mframe = monthly_frame(df, day)
    xcol = 'pick_day' if day else 'pick_hour'
    xlabel = {'pick_day': 'Deň', 'pick_hour': 'Hodina'}
    xticks = {'pick_day': list(range(1, 32)), 'pick_hour': list(range(24))}
    pass_fares_plot = px.bar(mframe, x=xcol, y=['Cestujúci', 'Jazdy'], barmode='group',
                             labels={xcol: xlabel[xcol], 'value': 'Hodnoty', 'variable': 'Premenná'})
    pass_fares_plot.update_layout(xaxis=dict(tickmode='array', tickvals=xticks[xcol]))
    return pass_fares_plot

In [ ]:
monthly_plot('Podľa dní') #  facina, samozrejme

#### Hore vyrobene dve funkcie hodime do suboru `data_funkcie.py`. V dalsich NB ich budeme odtial importovat.
#### Nebolo by odveci, mať graf aj pre cestujúcich, jazdy podľa dní v týždni.

In [ ]:
# nie je to v poriadku, nie kazdy den v tyzdni je v januari 2015 rovnaky pocet raz
# chceme pocty za jeden pondelok, utorok, ...
# klucove - aky den bol 1. januara 2015?
# datetime ma funkciu weekday 
from datetime import date
print(date(2015, 1, 1).weekday()) # 0 je tu pondelok, ... 6 je nedela - nie ako v polars

In [ ]:
# teda 1. jan. 2015 bol stvrtok a pocty dni v januari su
# 1, 8, 15, 22, 29 - stvrtky, 30-pia, 31-so bude po 5, ostatne dni po 4
pocty = np.array([4, 4, 4, 5, 5, 5, 4])  # preco nie len list?

In [ ]:
# zosumarizujeme, vyrobime funkcie week_plot, view_month_week a hodime ich do data_funkcie.py
def week_plot(frm):
    groupped = frm.groupby(frm['pick_dt'].dt.weekday)
    df_weekday = pd.DataFrame(groupped['passengers'].count())
    df_weekday['pick_day'] = df_weekday.index + 1
    df_weekday = df_weekday.rename(columns={'passengers': 'pass_count'})                    
    pocty = np.array([4, 4, 4, 5, 5, 5, 4])
    print(df_weekday.columns)
    df_weekday['pass_count'] = df_weekday['pass_count'] / pocty
    graf = px.bar(df_weekday, x='pick_day', y='pass_count', barmode='group', width=750, height=400)
    xtext = ['Pondelok', 'Utorok', 'Streda', 'Štvrtok', 'Piatok', 'Sobota', 'Nedeľa']
    graf.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 8)), title='Deň v týždni',
                       ticktext=xtext, tickangle=0), yaxis=dict(title="Priem. počet cestujúcich"))
    return graf

# v aplikacii budu tie tri grafy - mesacne podla hodin, dni a tyzdenny v jednej 'zalozke'

def view_month_week(doh):
    if doh in ['Podľa dní', 'Podľa hodín']:
        return monthly_plot(doh)
    return week_plot(df)

In [ ]:
view_month_week('frc')